<a href="https://colab.research.google.com/github/slinkierchip/Identificacion-del-lenguaje-de-se-as-americanas/blob/main/Identificacion_de_manos_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import os
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image as image_utils
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import load_model
import itertools
from sklearn.metrics import confusion_matrix

Mounted at /content/drive


In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download grassknoted/asl-alphabet
! unzip /content/asl-alphabet.zip

In [ ]:
K.clear_session()

data_entrenamiento = '/content/asl_alphabet_train/asl_alphabet_train'
data_validacion = '/content/asl_alphabet_test'

epocas=15
longitud, altura = 200, 200
batch_size = 32

Conv1 = 32
Conv2 = 64


tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)

tamano_pool = (2, 2)
clases = 29
lr = 0.1

In [ ]:
validation_ds = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale= 1./255,   #Pasar los pixeles de 0 a 255 | 0 a 1
    shear_range = 0.3, #Generar nuestras imagenes inclinadas para un  mejor entrenamiento
    zoom_range = 0.3,  #Genera imagenes con zoom para un mejor entrenamiento
    horizontal_flip=True, #Invierte las imagenes para mejor entrenamiento
    validation_split=0.1
)


val=validation_ds.flow_from_directory(
    data_entrenamiento,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset='training'
)

train=validation_ds.flow_from_directory(
    data_entrenamiento,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset='validation'

) 

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [ ]:
cnn=tf.keras.models.Sequential()

cnn.add(Convolution2D(Conv1, tamano_filtro1, padding='same', input_shape=(altura, longitud,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(Conv2, tamano_filtro2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten()) 

cnn.add(Dense(1500,activation='relu')) 
cnn.add(Dropout(0.5)) 
cnn.add(Dense(clases, activation='softmax'))

cnn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
cnn.fit(val, epochs=epocas, validation_data=train)

cnn.save('/content/drive/MyDrive/modelo.h5')
cnn.save_weights('/content/drive/MyDrive/pesos.h5')

Epoch 1/15
2447/2447 [==============================] - 9739s 4s/step - loss: 2.1090 - accuracy: 0.3795 - val_loss: 1.7270 - val_accuracy: 0.3999
Epoch 2/15
1045/2447 [===========>..................] - ETA: 1:31:32 - loss: 1.0771 - accuracy: 0.6455

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-f903c3bab267>", line 16, in <module>
    cnn.fit(val, epochs=epocas, validation_data=train)
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1384, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 915, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 947, in _call
    retu

KeyboardInterrupt: ignored